In [9]:
# > miio discover
# Device ID: 260426251
# Model info: roborock.vacuum.s5
# Address: 192.168.0.235
# Token: 793477645152e4839584f77617xxxxx via stored token


import socket

UDP_IP1 = "192.168.0.235"
token=bytes.fromhex("793477645152e4839584f77617xxxxx")
UDP_PORT = 54321          # used port on external device

port = 52129
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  
s.bind(("", port))
    
def SendData(hex_message:bytes):
    msg = hex_message
    s.sendto(msg, (UDP_IP1, UDP_PORT))
    return s

def ReceiveData(sock):
    print("Receiving data: \n")

    data, addr = sock.recvfrom(4096)      # buffer size is 4096 bytes
    return data


OSError: [Errno 98] Address already in use

In [41]:
# handshake
import datetime
handshake = bytes.fromhex("21310020ffffffffffffffffffffffffffffffffffffffffffffffffffffffff")
SendData(handshake)
hs_response = ReceiveData(s)
hex_response = hs_response.hex()
_device_id = hs_response[8:12]
_device_ts = datetime.datetime.utcfromtimestamp(int(hex_response[24:32],16))
print ("time delta: " + str(datetime.datetime.utcnow() - _device_ts))
str(_device_ts)
id = 0


Receiving data: 

18793 days, 23:19:19.367367


'1970-01-02 19:29:17'

In [43]:
from miio.protocol import Message
import json



In [44]:
send_ts = _device_ts + datetime.timedelta(seconds=4)
id += 1

header = {
    "length": 0,
    "unknown": 0x00000000,
    "device_id": _device_id,
    "ts": send_ts,
}
request = {'id': id, 'method': 'set_tar_temp', 'params': [20]}
msg = {"data": {"value": request}, "header": {"value": header}, "checksum": 0}
m = Message.build(msg, token=token)
#_LOGGER.debug("%s:%s >>: %s", self.ip, self.port, request)

m2 = Message.parse(m, token=token)


In [45]:
SendData(m)
m3 = Message.parse(ReceiveData(s), token=token)
m3.data.value

Receiving data: 



{'error': {'code': -32601, 'message': 'Method not found.'}, 'id': 2}

In [59]:
p = "2131005000000000037439e60000108b771322cf1c39dfe62b2de9bb472ccb4562fdb3fcc6f7022ebdc48397b6bdabd934fcb50ca7ea7fdc090de3462fb0014684a33abc4ab37991ddfcac81be47b337"

m = Message.parse( bytes.fromhex(p), token=token)
m.data.value

{'id': 4, 'method': 'set_power', 'params': ['off']}

In [156]:
data = bytes.fromhex("7b226964223a322c226d6574686f64223a226170705f7374617274227d")

In [157]:
data

b'{"id":2,"method":"app_start"}'

In [167]:
header = {
    "length": 0,
    "unknown": 0x00000000,
    "device_id": _device_id,
    "ts": send_ts,
}
msg = {"data": {"value": json.loads(data)}, "header": {"value": header}, "checksum": 0}


Message.build(msg, token = token).hex()[32:]

'91cffb752d05e325045659f06f0c5b3d50d586207d7cc8198c1d2c8a3941cb39372c771a652f083af48958da136424bfad85ca016240d409eff4d9a68f0870af'

In [65]:
# AP Partner packet generator
def ppaa(val:str):
    return {
        'prefix':val[0:9],
        'po': val[10],
        'mo': val[11],
        'wi': val[12],
        'sw': val[13],
        'tt': val[14:16],
        'li': val[16],
        'suffix': val[17]
    }

ppaa("0100007411143114A1") # fail

{'prefix': '010000741',
 'po': '1',
 'mo': '4',
 'wi': '3',
 'sw': '1',
 'tt': '14',
 'li': 'A',
 'suffix': '1'}